In [1]:
# Libs
from bs4 import BeautifulSoup
import glob
import os
import pandas as pd
import requests

In [2]:
def postnl_scrape(cep):
    """
        This function shall request coordinates from postcodebijadres
    """
    # creates link
    url = 'https://postcodebijadres.nl/' + cep
    # create header to simulate human behaviour on scrape
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"}
    # scrape
    soup = BeautifulSoup(requests.get(
        url, headers=headers).content, "html.parser")
    # html as string
    return soup

In [3]:
df_zipcode = pd.read_csv('../data/raw/zipcode.csv', index_col=[0])
df_housing = pd.read_csv("../data/processed/all_content.csv", index_col=[0])
df_housing['postcode'] = df_housing['location'].str[:7].str.replace(" ","")

In [4]:
# what are the zipcodes we have scrapped?
temp1 = df_housing['postcode'].unique()
# what are those we already have the coordinates?
temp2 = df_zipcode['postcode'].unique()
# what is new?
cep_zip = list(set(temp1) - set(temp2))
# how many new zipcodes without coordidates we have
print(len(cep_zip))

27


In [5]:
# empty variables
zipcode = pd.DataFrame({})
description_1 = {}

In [6]:
# read each new zipcode
for cep in cep_zip:
    # load the scraper
    soup = postnl_scrape(cep)
    try:
        # get first table
        split0 = soup.find_all(
            'table', {"class": "table table-bordered mt-3"})[0]
        # get second table
        split1 = soup.find_all(
            'table', {"class": "table table-bordered mt-3"})[1]
        # coordinates
        lat = split1.find_all('td')[1].text
        lon = split1.find_all('td')[3].text
        # save on dict
        description_1.update({'latitude': lat, 'longitude': lon})
        # each row of the table
        for num in range(9):
            # get key and value
            split0_key = split0.find_all('th')[num].text
            split0_val = split0.find_all('td')[num].text
            # save on dict
            description_1.update({split0_key: split0_val})
        # dict into df
        zipcode_ = pd.DataFrame(description_1, index=[0])
        # add on the df
        zipcode_['postcode'] = zipcode_['Postcode'].str.replace(" ","")
        df_zipcode = pd.concat([df_zipcode, zipcode_], 0)
    except:
        print('#####################################', cep)
        pass

/var/folders/pl/wm7wjyvn1vvflk70yn1b6n6c0000gn/T/ipykernel_94915/3688265986.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_zipcode = pd.concat([df_zipcode, zipcode_], 0)


##################################### 3824KB
##################################### 3824KC
##################################### 3824KG
##################################### 3118JR


In [7]:
# add new info into the housing dataframe
zipcode_housing_coo = pd.merge(df_housing, df_zipcode, how='left', on='postcode')

In [8]:
col = (map(lambda x: x.lower(), zipcode_housing_coo.columns))
col = list(col)
zipcode_housing_coo.columns = col

In [9]:
zipcode_housing_coo = zipcode_housing_coo.rename(columns={"plaats":"city"})

In [10]:
# saving data
zipcode_housing_coo.to_csv('../data/processed/df_housing.csv')
df_zipcode.to_csv('../data/raw/zipcode.csv')